# Recording a Gesture for Shimi

First, run the cell below to instatiate the shimi object. It should move to its initial positions and then relax.

In [1]:
# Import some matplolib shortcuts for Jupyter notebook
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from shimi import *
from motion.move import *
from config.definitions import *
from motion.recorder import *
from motion.playback import *
from audio.audio import *
from posenet.posenet import *

import time
import datetime

from copy import deepcopy

import os

import threading

# Load Shimi
shimi = Shimi()

local_gestures = {}

Connecting on /dev/tty.usbmodem145101
Found motors with the following IDs: [1, 2, 3, 4, 5]
(OrderedDict([('model', 'MX-28'),
              ('firmware', 30),
              ('id', 1),
              ('baudrate', 1),
              ('return delay time', 0),
              ('angle limit', (-99.03, 14.64)),
              ('highest temperature limit', 80.0),
              ('voltage limit', (6.0, 16.0)),
              ('max torque', 100.0),
              ('status return level', 'always'),
              ('alarm LED', ('Overload Error', 'Overheating Error')),
              ('alarm shutdown', ('Overload Error', 'Overheating Error')),
              ('torque_enable', False),
              ('LED', False),
              ('pid gain', (0.0, 0.0, 4.0)),
              ('goal position', 15.25),
              ('moving speed', 0.0),
              ('torque limit', 100.0),
              ('present position', 15.34),
              ('present speed', 0.0),
              ('present load', 0.0),
              ('presen

The cell below defines the recorder object. The first parameter is a reference to the motor controller, the second is a **list** of motor ids, referenced by a convenience property on `shimi`. The options are as follows:
* `shimi.torso`
* `shimi.neck_lr`
* `shimi.neck_ud`
* `shimi.phone`
* `shimi.foot`
* `shimi.all_motors`: this **returns a list**, so you don't need to wrap it in one in this call


The final parameter is the duration for which you want to record.

In [ ]:
r = Recorder(shimi, shimi.all_motors, 8.0)

This starts a recording. It will count down from 3 to let you know when the recording starts, then move Shimi as you want it to move. **The first thing this call does is relax the motors, so be sure it doesn't collapse on itself when you call this.**

In [ ]:
r.record()
r.plot(plt.axes())

Then, run this to play back the recorded gesture.

In [ ]:
r.play(plt.axes())

### Here are some helper functions.

In [ ]:
# Puts shimi in a neutral position with motors non-compliant
shimi.initial_position()

## **Make sure Shimi will not collapse before you run this!!!**

In [ ]:
# Makes the motors compliant
shimi.disable_torque()

### Here's a hard-coded demo of Shimi dancing to Hey Ya

In [ ]:
play_outkast(shimi)

In [2]:
play_opera(shimi)

Setting positions {1: 8.285927600742228, 3: -1.7472456291869554, 2: -23.18261737879165, 4: -12.350266888364938, 5: 5.1400000000001365}
Didn't sleep.
Setting positions {1: 9.206741177801279, 3: -3.936710942120143, 2: -20.943156051713334, 4: -12.351439705300397, 5: 5.140000000000137}
Setting positions {5: 5.140000000000135}
Setting positions {5: 5.140000000000136}
Didn't sleep.
Setting positions {5: 5.140000000000133}
Setting positions {5: 5.140000000000134}
Setting positions {5: 5.140000000000132}
Setting positions {5: 5.140000000000134}
Setting positions {3: -2.1051441193455425}
Setting positions {5: 5.140000000000129}
Setting positions {1: -50.211288040615784}
Setting positions {5: 5.140000000000131}
Setting positions {5: 5.140000000000129}
Setting positions {5: 5.14000000000013}
Setting positions {5: 5.140000000000129}
Setting positions {5: 5.14000000000013}
Setting positions {5: 5.140000000000128}
Setting positions {5: 5.1400000000001285}
Setting positions {5: 5.140000000000127}
Set

Setting positions {5: 5.140000000000074}
Setting positions {5: 5.140000000000075}
Setting positions {5: 5.140000000000073}
Setting positions {5: 5.140000000000074}
Setting positions {5: 5.1400000000000725}
Setting positions {5: 5.140000000000074}
Setting positions {5: 5.1400000000000725}
Setting positions {5: 5.140000000000074}
Setting positions {5: 5.1400000000000725}
Setting positions {5: 5.140000000000073}
Setting positions {5: 5.140000000000072}
Setting positions {5: 5.1400000000000725}
Setting positions {5: 5.140000000000071}
Setting positions {5: 5.140000000000072}
Setting positions {5: 5.14000000000007}
Setting positions {5: 5.140000000000072}
Setting positions {5: 5.14000000000007}
Setting positions {5: 5.140000000000071}
Setting positions {5: 5.140000000000069}
Setting positions {5: 5.140000000000071}
Setting positions {5: 5.140000000000069}
Setting positions {5: 5.14000000000007}
Setting positions {5: 5.140000000000069}
Setting positions {5: 5.140000000000071}
Setting positio

Setting positions {3: -4.884200318836393, 5: 5.140000000000031}
Setting positions {5: 5.140000000000029}
Setting positions {5: 5.14000000000003}
Setting positions {5: 5.140000000000029}
Setting positions {5: 5.14000000000003}
Setting positions {5: 5.140000000000029}
Setting positions {5: 5.14000000000003}
Setting positions {5: 5.140000000000027}
Setting positions {5: 5.14000000000003}
Setting positions {5: 5.140000000000028}
Setting positions {5: 5.140000000000029}
Setting positions {5: 5.140000000000027}
Setting positions {5: 5.140000000000029}
Setting positions {5: 5.140000000000027}
Setting positions {5: 5.140000000000029}
Setting positions {5: 5.140000000000027}
Setting positions {5: 5.140000000000028}
Setting positions {5: 5.140000000000027}
Setting positions {5: 5.140000000000028}
Setting positions {5: 5.140000000000027}
Setting positions {3: -34.04504656112678, 5: 5.140000000000028}
Setting positions {5: 5.140000000000027}
Setting positions {5: 5.140000000000028}
Setting positio

Setting positions {5: 5.140000000000007}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000007}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000007}
Setting positions {5: 5.140000000000005}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000005}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000004}
Setting positions {5: 5.140000000000006}
Didn't sleep.
Setting positions {5: 5.140000000000005}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000004}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000004}
Setting positions {1: 0.38282964031359457}
Setting positions {5: 5.140000000000005}
Setting positions {5: 5.140000000000004}
Setting positions {5: 5.140000000000005}
Setting positions {5: 5.140000000000004}
Setting positions {5: 5.140000000000006}
Setting positions {5: 5.140000000000003}
Setting positions {5: 5.140000000000005}


Setting positions {5: 5.14}
Setting positions {5: 5.139999999999999}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.139999999999999}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.139999999999999}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.14}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.14}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.14}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.139999999999999}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.14}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.14}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.139999999999999}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.139999999999999}
Setting positions {5: 5.140000000000001}
Setting positions {1: 5.440623138968517, 5: 5.14}
Setting positions {5: 5.140000000000001}
Setting positions {5: 5.14}
Setting position

In [ ]:
p = PoseNet(shimi)

In [ ]:
p.record()

In [ ]:
p.play(None, plt.axes())

In [ ]:
p.stop_posenet()

In [ ]:
m = LinearMove(shimi, shimi.torso, 1.0, 1.0, normalized_positions=True)
m.start()

In [ ]:
playback(shimi, r.motors, r.dur, r.timestamps, r.positions, r.velocities, plt.axes())

In [ ]:
r.append_recording(r1)

In [ ]:
r.save("da_da_da")

In [ ]:
r = load_recorder(shimi, "opera")

In [ ]:
r.append_recording(load_recorder(shimi, "da_da_da"))